Привет! В этом ноутбуке мы будем предсказывать климат методом random forest

In [40]:
import pandas as pd
import numpy as np
from sklearn.tree import export_graphviz
from sklearn.metrics import roc_auc_score
import pydot
import matplotlib.pyplot as plt

In [13]:
df = pd.read_csv('temps.csv')
df.head(10)

,year,month,day,week,temp_2,temp_1,average,actual,friend
0,2019,1,1,Fri,45,45,45.6,45,29
1,2019,1,2,Sat,44,45,45.7,44,61
2,2019,1,3,Sun,45,44,45.8,41,56
3,2019,1,4,Mon,44,41,45.9,40,53
4,2019,1,5,Tues,41,40,46.0,44,41
5,2019,1,6,Wed,40,44,46.1,51,40
6,2019,1,7,Thurs,44,51,46.2,45,38
7,2019,1,8,Fri,51,45,46.3,48,34
8,2019,1,9,Sat,45,48,46.4,50,47
9,2019,1,10,Sun,48,50,46.5,52,49


In [14]:
df.info() #данные корректные, менять не нужно

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 348 entries, 0 to 347
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   year     348 non-null    int64  
 1   month    348 non-null    int64  
 2   day      348 non-null    int64  
 3   week     348 non-null    object 
 4   temp_2   348 non-null    int64  
 5   temp_1   348 non-null    int64  
 6   average  348 non-null    float64
 7   actual   348 non-null    int64  
 8   friend   348 non-null    int64  
dtypes: float64(1), int64(7), object(1)
memory usage: 24.6+ KB


In [15]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
year,348.0,2019.000000,0.000000,2019.0,2019.000,2019.0,2019.000,2019.0
month,348.0,6.477011,3.498380,1.0,3.000,6.0,10.000,12.0
day,348.0,15.514368,8.772982,1.0,8.000,15.0,23.000,31.0
temp_2,348.0,62.652299,12.165398,35.0,54.000,62.5,71.000,117.0
temp_1,348.0,62.701149,12.120542,35.0,54.000,62.5,71.000,117.0
average,348.0,59.760632,10.527306,45.1,49.975,58.2,69.025,77.4
actual,348.0,62.543103,11.794146,35.0,54.000,62.5,71.000,92.0
friend,348.0,60.034483,15.626179,28.0,47.750,60.0,71.000,95.0


Перекодируем фичи в onehot формат

In [16]:
df_dum = pd.get_dummies(df)
df_dum.head()

,year,month,day,temp_2,temp_1,average,actual,friend,week_Fri,week_Mon,week_Sat,week_Sun,week_Thurs,week_Tues,week_Wed
0,2019,1,1,45,45,45.6,45,29,1,0,0,0,0,0,0
1,2019,1,2,44,45,45.7,44,61,0,0,1,0,0,0,0
2,2019,1,3,45,44,45.8,41,56,0,0,0,1,0,0,0
3,2019,1,4,44,41,45.9,40,53,0,1,0,0,0,0,0
4,2019,1,5,41,40,46.0,44,41,0,0,0,0,0,1,0


In [19]:
df_dum.shape

(348, 15)

Разбиваем данные на обучение и проверку

In [21]:
df_dum.columns

Index(['year', 'month', 'day', 'temp_2', 'temp_1', 'average', 'actual',
       'friend', 'week_Fri', 'week_Mon', 'week_Sat', 'week_Sun', 'week_Thurs',
       'week_Tues', 'week_Wed'],
      dtype='object')

Будем предсказывать actual

In [31]:
from sklearn.model_selection import train_test_split

In [23]:
labels = df['actual']

df_dum = df_dum.drop('actual', axis = 1)

In [54]:
X_train, X_test, y_train, y_test = train_test_split(df_dum,
                                                   labels,
                                                   test_size = 0.20,
                                                   random_state = 42)

In [58]:
print('train features shape:', X_train.shape)
print('train labels shape:', y_train.shape)
print('test features shape:', X_test.shape)
print('test labels shape:', y_test.shape)

train features shape: (278, 14)
train labels shape: (278,)
test features shape: (70, 14)
test labels shape: (70,)


In [32]:
from sklearn.ensemble import RandomForestRegressor

In [59]:
rf = RandomForestRegressor(n_estimators= 1000, random_state=42)

rf.fit(X_train, y_train);

Для начала узнаем отклонение значений предсказаний от тестовой выборки Actual

In [91]:
y_pred = rf.predict(X_test)
errors = np.mean(abs(predictions - y_test))

print('Средняя абсолютная ошибка, отклонение на:', round(errors, 2), 'градусов от тестовой выборки')

Средняя абсолютная ошибка, отклонение на: 3.78 градусов от тестовой выборки


Оценим точность модели вручную

In [86]:
error_prob = np.mean(100*(errors / y_test))

accuracy = 100 - error_prob

print('Точность:', round(accuracy, 2), '%.')

Точность: 94.02 %.


Точность модели высока, можем считать модель работоспособной.